# Self Query Retriever Test Notebook

In [1]:
import os
import shutil
from loguru import logger
from intellitube.utils import ChatManager
from intellitube.vector_store import VectorStoreManager

## Create Vector Store

In [2]:
vs_path = "test_data/qdrant_vector_store/test_vct_store"
vsc_path = os.path.join(vs_path, "collections")

for path in [vsc_path, vs_path]:
    if os.path.exists(path):
        logger.info(f"Removing path: {path}")
        shutil.rmtree(path)

vectorstore = VectorStoreManager(
    path_on_disk=vs_path,
    collection_path_on_disk=vsc_path
)

2025-07-12 16:40:13.514 | INFO     | __main__:<module>:6 - Removing path: test_data/qdrant_vector_store/test_vct_store
2025-07-12 16:40:24.033 | DEBUG    | intellitube.vector_store:init_vector_store:73 - Creaing Client...
2025-07-12 16:40:24.245 | DEBUG    | intellitube.vector_store:init_vector_store:81 - Creaing vector store


## Loading YouTube Video Transcript

In [3]:
from intellitube.utils import (
    YTContentData, download_youtube_audio_or_transcript, webvtt_2_str
)

# Steve Jobs' 2005 Stanford Commencement Address
video_url = "https://www.youtube.com/watch?v=UF8uR6Z6KLc"
video_data: YTContentData = download_youtube_audio_or_transcript(video_url)
vtt_str = webvtt_2_str(vtt_file_path=video_data.transcript_path)

print(video_data, end='\n\n')
print(vtt_str[:500])

2025-07-12 16:40:24.259 | DEBUG    | intellitube.utils.youtube:download_youtube_audio_or_transcript:133 - Cache exists, validating cache...
2025-07-12 16:40:24.260 | DEBUG    | intellitube.utils.youtube:download_youtube_audio_or_transcript:145 - Cache contains the requested data. Using cache.


type='text' transcript_path='test_data/cache/youtube/downloads/0290de54-13f5-4bbe-b3de-7b06f46a5f07.vtt' audio_path=None

This program is brought to you by Stanford University.
Please visit us at stanford.edu
Thank You. I am honored to be with you today at your commencement
from one of the finest universities in the world.
Truth be told I never graduated from college
and this is the closest I've ever gotten to a college graduation.
Today I want to tell you three stories from my life. That's it.
No big deal. Just three stories.
The first story is about connecting the dots.
I dropped out of Reed College after the fir


## Adding to Vector Database

### Split Text

In [4]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=256,
)
texts = text_splitter.split_text(vtt_str)

print(len(texts), end="\n" + "-"*20 + "\n\n")
print(texts[0], end="\n" + "-"*20 + "\n\n")

documents = []
for i, chunk in enumerate(texts):
    document = Document(
        page_content=chunk,
        metadata={
            "source": video_url,
            "chunk_index": i + 1,
        }
    )
    documents.append(document)

47
--------------------

This program is brought to you by Stanford University.
Please visit us at stanford.edu
Thank You. I am honored to be with you today at your commencement
from one of the finest universities in the world.
Truth be told I never graduated from college
and this is the closest I've ever gotten to a college graduation.
Today I want to tell you three stories from my life. That's it.
No big deal. Just three stories.
The first story is about connecting the dots.
I dropped out of Reed College after the first 6 months,
--------------------



In [5]:
vectorstore.add_documents(
    documents, split_text=False,
    skip_if_collection_exists=True,
)

2025-07-12 16:40:24.352 | INFO     | intellitube.vector_store:add_documents:108 - Adding documents...


In [ ]:
# vectorstore.retriever.

## Creating the retriever

In [6]:
retriever = vectorstore.vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.6}
)